In [2]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [4]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [5]:
import pandas as pd
import json
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Cell_Phones_and_Accessories_5.json.gz')

In [6]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,NaN,NaN
1,5.0,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,NaN,NaN
2,3.0,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,NaN,NaN
3,2.0,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,NaN,NaN
4,4.0,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,NaN,NaN


In [50]:
df100k = df.head(100000)
df100k

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"08 4, 2014",A24E3SXTC62LJI,7508492919,{'Color:': ' Bling'},Claudia Valdivia,Looks even better in person. Be careful to not...,Can't stop won't stop looking at it,1407110400,NaN,NaN
1,5.0,True,"02 12, 2014",A269FLZCB4GIPV,7508492919,NaN,sarah ponce,When you don't want to spend a whole lot of ca...,1,1392163200,NaN,NaN
2,3.0,True,"02 8, 2014",AB6CHQWHZW4TV,7508492919,NaN,Kai,"so the case came on time, i love the design. I...",Its okay,1391817600,NaN,NaN
3,2.0,True,"02 4, 2014",A1M117A53LEI8,7508492919,NaN,Sharon Williams,DON'T CARE FOR IT. GAVE IT AS A GIFT AND THEY...,CASE,1391472000,NaN,NaN
4,4.0,True,"02 3, 2014",A272DUT8M88ZS8,7508492919,NaN,Bella Rodriguez,"I liked it because it was cute, but the studs ...",Cute!,1391385600,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
99995,5.0,True,"04 4, 2013",ADOGGEGPTDOBW,B006LP14Z8,NaN,T. Harris,"Great little hard cover for my son's ""dumb"" ph...",Easy to use,1365033600,NaN,NaN
99996,4.0,True,"08 31, 2012",A1DQZ1II957I6A,B006LP14Z8,NaN,KL,This is a good case that holds my phone togeth...,Good Case,1346371200,NaN,NaN
99997,5.0,True,"10 24, 2012",AS9YSOCMP1D9D,B006LQ37BQ,NaN,Gypsy JR,"I initially thought it wasn't charging, since ...",Works well...........,1351036800,NaN,NaN
99998,5.0,False,"03 12, 2012",A2LMWI9IUKXUKB,B006LRP00G,NaN,raw2000j,Charger is great quality and charges both rath...,Great!,1331510400,NaN,NaN


In [51]:
df100k = df100k.rename({'reviewText': 'review'}, axis=1)

In [52]:
df100k['label'] = df100k['overall'].apply(lambda c: 1 if c >3 else (0 if c<3 else 'neut'))

In [53]:
df100k = df100k.drop(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style', 'reviewerName', 'summary', 'unixReviewTime', 'vote', 'image'], axis=1)

In [54]:
df100k = df100k[df100k['label'] != 'neut']

In [55]:
def get_top_data(top_n):
    top_data_df_positive = df100k[df100k['label'] == 1].head(top_n)
    top_data_df_negative = df100k[df100k['label'] == 0].head(top_n)
    top_data_df_small = pd.concat([top_data_df_positive, top_data_df_negative])
    return top_data_df_small

# Function call to get the top 10000 from each sentiment
top_data_df_small = get_top_data(top_n=15000)

# After selecting top few samples of each sentiment
print("After segregating and taking equal number of rows for each sentiment:")
print(top_data_df_small['label'].value_counts())
top_data_df_small.head(10)

After segregating and taking equal number of rows for each sentiment:
1    15000
0    14849
Name: label, dtype: int64


,review,label
0,Looks even better in person. Be careful to not...,1
1,When you don't want to spend a whole lot of ca...,1
4,"I liked it because it was cute, but the studs ...",1
7,It is a very cute case. None of the jewels hav...,1
9,I really love this case... you have to keep yo...,1
10,its super cute and makes my phone pretting and...,1
11,Another great product that my daughter she use...,1
13,I purchased this for my grand-daughters phone....,1
14,Beautiful quality and outstanding product! Eve...,1
15,It is such a good case for a low price. I have...,1


In [56]:
train, test = train_test_split(top_data_df_small, test_size=0.3, random_state=42, shuffle=True)

In [57]:
train = train.rename({'review': 'DATA_COLUMN', 'label': 'LABEL_COLUMN'}, axis=1)
train.head()

,DATA_COLUMN,LABEL_COLUMN
16670,"Like the convenient, retractable cord.",1
99335,"The color is nice, but it didn't fit my 4S wel...",0
29981,"The cord pulls away from the plug quickly, and...",0
8354,Item not as described! The description says t...,0
13217,I purchased this product about two years ago a...,1


In [58]:
train['DATA_COLUMN'].isnull().sum().sum()

3

In [59]:
train['DATA_COLUMN'].fillna("", inplace = True)

In [60]:
train['DATA_COLUMN'].isnull().sum().sum()

0

In [61]:
test = test.rename({'review': 'DATA_COLUMN', 'label': 'LABEL_COLUMN'}, axis=1)
test.head()

,DATA_COLUMN,LABEL_COLUMN
37899,this didn't work as well as I was hoping it wo...,0
17302,I highly recommend this case to anyone that ha...,1
17846,The SSD may be one of the most inexpensive one...,1
13632,When I had my Kindle (before I sold it to get ...,1
8432,"great price, holds phone snugly, fits well on ...",1


In [62]:
test['DATA_COLUMN'].isnull().sum().sum()

0

## BERT IMPLEMENTATION STARTS HERE 

### This function will accept our train and test datasets and convert each row into an InputExample object. 

In [63]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')

### This function will tokenize the InputExample objects, then create the required input format with the tokenized objects, finally, create an input dataset that we can feed to the model. 

In [64]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

### Calling above functions 

In [65]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

C:\Users\haris\anaconda3\envs\bert\lib\site-packages\transformers\tokenization_utils_base.py:2068: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


## Configuring the BERT model and Fine-tuning 

In [66]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
   1306/Unknown - 34774s 27s/step - loss: 0.2579 - accuracy: 0.8891WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
1306/1306 [==============================] - 36537s 28s/step - loss: 0.2579 - accuracy: 0.8892 - val_loss: 0.2015 - val_accuracy: 0.9367
Epoch 2/2
1306/1306 [==============================] - 32715s 25s/step - loss: 0.0662 - accuracy: 0.9783 - val_loss: 0.2627 - val_accuracy: 0.9353


### Saving the model 

In [68]:
model.save('bert_model')

INFO:tensorflow:Assets written to: bert_model\assets


INFO:tensorflow:Assets written to: bert_model\assets


### Loading the saved model 

In [70]:
from tensorflow import keras

In [72]:
new_model = keras.models.load_model('bert_model')
new_model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________
